In [42]:
import pickle
import numpy as np
from tqdm import tqdm
from functools import partial
import pandas as pd
from konlpy.tag import Komoran
import os
from gensim.models import Word2Vec

In [96]:
max_n = 2
total_data = [[], [], [], [], []]
for i in range(1, max_n + 1):
    file_name = 'data_256_cat%s.pkl' %i
    loaded_data = pickle.load(open(file_name, 'rb'))

    _image_data, _sentence_data, _sentence_tags, _hashtag_data, _metadata = loaded_data[1]

    _sentence_tags = np.array(_sentence_tags)
    _hashtag_data = np.array(_hashtag_data)
    
    total_data[0].append(_image_data)
    total_data[1].append(_sentence_data)
    total_data[2].append(_sentence_tags)
    total_data[3].append(_hashtag_data)
    total_data[4].append(_metadata)

In [97]:
_image_data = np.concatenate(total_data[0])
_sentence_data = np.concatenate(total_data[1])
_sentence_tags = np.concatenate(total_data[2])
_hashtag_data = np.concatenate(total_data[3])
_metadata = pd.concat(total_data[4])

In [98]:
np.random.seed(1234)
n_valid = 700
n_train = _image_data.shape[0] - n_valid
n_total = n_train + n_valid
indices = np.random.choice(np.arange(_image_data.shape[0]), n_total, replace=False)
train_idx = indices[:n_train]
valid_idx = indices[n_train:]

In [99]:
image_data = _image_data[train_idx]
sentence_data = _sentence_data[train_idx]
sentence_tags = _sentence_tags[train_idx]
hashtag_data = _hashtag_data[train_idx]
metadata = _metadata.iloc[train_idx]

In [100]:
image_valid = _image_data[valid_idx]
sentence_valid = _sentence_data[valid_idx]
sentence_tags_valid = _sentence_tags[valid_idx]
hashtag_valid = _hashtag_data[valid_idx]
metadata_valid = _metadata.iloc[valid_idx]

In [101]:
print(image_data.shape)
print(sentence_data.shape)
print(sentence_tags.shape)
print(hashtag_data.shape)
print(metadata.shape)

(13300, 256, 256, 3)
(13300,)
(13300,)
(13300,)
(13300, 15)


In [21]:
komoran = Komoran()
train_tags = np.array([[t[0] for t in komoran.pos(s)] for s in tqdm(sentence_data)])

100%|██████████| 34300/34300 [00:43<00:00, 791.94it/s] 


In [25]:
for tags in train_tags:
    tags.insert(0, '\t')
    tags.append('\n')

In [22]:
valid_tags = np.array([[t[0] for t in komoran.pos(s)] for s in tqdm(sentence_valid)])

100%|██████████| 700/700 [00:00<00:00, 2386.48it/s]


In [102]:
max_seq_length = max(map(len, sentence_data))

In [134]:
output_chars = set()
for s in sentence_data:
    for c in s:
        if c not in output_chars:
            output_chars.add(c)

output_chars.add('<PAD>')
output_chars = sorted(list(output_chars))
n_chars = len(output_chars)
output_dict = {c: i for i, c in enumerate(output_chars)}

dec_input_data = np.zeros(shape=(sentence_data.shape[0], max_seq_length, n_chars))
dec_output_data = np.zeros(shape=(sentence_data.shape[0], max_seq_length))

for i, s in enumerate(sentence_data):
    for j, c in enumerate(s):
        dec_input_data[i, j, output_dict[c]] = 1.
        if j > 0:
            dec_output_data[i, j-1] = output_dict[c]
            
dec_input_valid = np.zeros(shape=(sentence_valid.shape[0], max_seq_length, n_chars))
dec_output_valid = np.zeros(shape=(sentence_valid.shape[0], max_seq_length))

dec_input_valid[:,0,output_dict['\t']] = 1.

for i, s in enumerate(sentence_valid):
    for j, c in enumerate(s):
        if j > 0:
            dec_output_valid[i, j-1] = output_dict[c if c in output_dict else '<PAD>']

In [137]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [138]:
tf.reset_default_graph()

learning_rate1 = 0.001
learning_rate2 = 0.001
n_hidden = 128

inputs = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
with tf.variable_scope('auto_encoder'):
    net = tf.layers.conv2d(inputs, 32, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d(net, 8, [5,5], strides=2, padding='SAME')
    middle = tf.layers.conv2d(net, 4, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d_transpose(middle, 8, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d_transpose(net, 32, [5,5], strides=2, padding='SAME')
    outputs = tf.layers.conv2d_transpose(net, 3, [5,5], strides=2, padding='SAME')

loss = tf.reduce_mean(tf.square(outputs - inputs))
train_op = tf.train.AdamOptimizer(learning_rate1).minimize(loss)

In [139]:
middle

<tf.Tensor 'auto_encoder/conv2d_2/BiasAdd:0' shape=(?, 32, 32, 4) dtype=float32>

In [140]:
enc_input = tf.reshape(middle, shape=(-1, 256, 32*32*4//256))
dec_input = tf.placeholder(tf.float32, shape=[None, max_seq_length, n_chars])
dec_output = tf.placeholder(tf.int32, shape=[None, max_seq_length])

with tf.variable_scope('generator_enc'):
    enc_cell = tf.nn.rnn_cell.GRUCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.8)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

with tf.variable_scope('generator_dec'):
    dec_cell = tf.nn.rnn_cell.GRUCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.8)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state=enc_states,
                                            dtype=tf.float32)
    
outputs = tf.layers.dense(outputs, n_chars, activation=None)
loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = outputs, labels = dec_output))
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator_*')
train_op2 = tf.train.AdamOptimizer(learning_rate2).minimize(loss2, var_list=var_list)
train_op2_fine = tf.train.AdamOptimizer(learning_rate2).minimize(loss2)

In [162]:
outputs

<tf.Tensor 'dense/BiasAdd:0' shape=(?, 256, 1865) dtype=float32>

In [141]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [142]:
batch_size = 700
n_epoch = 20
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op, loss], feed_dict={inputs:batch_img})
        total_loss += c
        
    c = sess.run(loss, feed_dict={inputs:image_valid})
    
    print('[Epoch: %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch: 1] cost = 8250.658, valid cost = 2996.501


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch: 2] cost = 2484.674, valid cost = 2216.202


  0%|          | 0/19 [00:00<?, ?it/s]

...

[Epoch: 18] cost = 590.525, valid cost = 621.280


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch: 19] cost = 590.208, valid cost = 610.762


100%|██████████| 19/19 [00:13<00:00,  1.37it/s]


[Epoch: 20] cost = 567.286, valid cost = 592.406


In [144]:
batch_size = 700
n_epoch = 50
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_in = dec_input_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_out = dec_output_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op2, loss2], feed_dict={inputs:batch_img, 
                                                       dec_input:batch_dec_in,
                                                       dec_output:batch_dec_out})
        total_loss += c
    
    c = sess.run(loss2, feed_dict={inputs:image_valid, 
                                   dec_input:dec_input_valid,
                                   dec_output:dec_output_valid})
    print('[Epoch %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch 1] cost = 4.710, valid cost = 4.729


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch 2] cost = 4.709, valid cost = 4.730


  0%|          | 0/19 [00:00<?, ?it/s]

...

[Epoch 18] cost = 4.649, valid cost = 4.734


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch 19] cost = 4.630, valid cost = 4.735


 21%|██        | 4/19 [00:08<00:32,  2.15s/it]


KeyboardInterrupt: 

In [148]:
batch_size = 700
n_epoch = 50
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_in = dec_input_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_out = dec_output_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op2_fine, loss2], feed_dict={inputs:batch_img, 
                                                       dec_input:batch_dec_in,
                                                       dec_output:batch_dec_out})
        total_loss += c
    
    c = sess.run(loss2, feed_dict={inputs:image_valid, 
                                   dec_input:dec_input_valid,
                                   dec_output:dec_output_valid})
    print('[Epoch %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch 1] cost = 2.924, valid cost = 1.751


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch 2] cost = 1.201, valid cost = 1.613


  0%|          | 0/19 [00:00<?, ?it/s]

...

[Epoch 48] cost = 0.710, valid cost = 2.311


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch 49] cost = 0.707, valid cost = 2.324


100%|██████████| 19/19 [00:35<00:00,  1.85s/it]


[Epoch 50] cost = 0.704, valid cost = 2.335


In [158]:
saver = tf.train.Saver()
saver.save(sess, './ver15')

'./ver15'

In [159]:
saver = tf.train.import_meta_graph('ver15.meta')

saver.restore(sess, tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./ver15


In [ ]:
    c = sess.run('dense/BiasAdd:0', feed_dict={inputs:image_valid, 
                                   dec_input:dec_input_valid,
                                   dec_output:dec_output_valid})

In [171]:
whos

Variable              Type                    Data/Info
-------------------------------------------------------
Komoran               type                    <class 'konlpy.tag._komoran.Komoran'>
Word2Vec              type                    <class 'gensim.models.word2vec.Word2Vec'>
batch                 int                     18
batch_dec_in          ndarray                 700x256x1865: 334208000 elems, type `float64`, 2673664000 bytes (2549.8046875 Mb)
batch_dec_out         ndarray                 700x256: 179200 elems, type `float64`, 1433600 bytes (1.3671875 Mb)
batch_img             ndarray                 700x256x256x3: 137625600 elems, type `uint8`, 137625600 bytes (131.25 Mb)
batch_size            int                     700
c                     float32                 2.3351858
dec_cell              DropoutWrapper          <tensorflow.python.keras.<...>object at 0x7f66fe4a5890>
dec_in_data           ndarray                 13300x256x1864: 6346547200 elems, type `float64`, 5

In [163]:
sess.run('dense/BiasAdd:0', feed_dict={''})

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,256,256,3]
	 [[node Placeholder (defined at <ipython-input-138-328e441e7177>:7) ]]
	 [[dense/BiasAdd/_887]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,256,256,3]
	 [[node Placeholder (defined at <ipython-input-138-328e441e7177>:7) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launc

In [166]:
def translate(image):
    dec_inp = np.zeros(shape=(1, max_seq_length, n_chars), dtype='float32')
    dec_inp[0,0,output_dict['\t']] = 1.
    result = sess.run('dense/BiasAdd:0', 
                      feed_dict={inputs: [image],
                                 dec_input: dec_inp})
    result = np.argmax(result, axis=2)
    decoded = [output_chars[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [172]:
pickle.dump([
        max_seq_length,
        n_chars,
        output_dict,
        output_chars
], open('seqlen_nchar_outdict_outchar.pkl', 'wb'))

In [170]:
tf.__version__

'2.3.1'

In [167]:
idx = 90
print(sentence_data[idx])
translate(image_data[idx])

	버버리 카톡문의 카드결제 가능 정품과 헷갈릴정도의 최상급만 취급 제품 출고영상 확인가능해요 배송기간 일 잡아주세요 풀구성 배송 피드에 없는 제품도 찾아드려요 제품문의는 프로필 상단 링크로 꾸욱



'오이     \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t'

In [79]:
from PIL import Image
import matplotlib.pyplot as plt

In [98]:
def translate(image):
    prediction = tf.argmax(outputs2, 2)
    dec_inp = np.zeros(shape=(1, max_seq_length, dict_len), dtype='float32')
    dec_inp[0,:,output_dict['\t']] = 1.
    # dec_out = np.eye(dict_len)[dec_out]
    result = sess.run(prediction, 
                      feed_dict={inputs: [image],
                                 dec_inputs: dec_inp})
    decoded = [output_chars[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [103]:
translate(image_data[50])

'오<UNK><UNK>'

In [80]:
start_idx, end_idx = 50, 100

results = sess.run(outputs2, feed_dict={inputs:image_data[start_idx:end_idx]})
print('정답 예측')

for l, p, img in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)], image_data[start_idx:end_idx]):
    print([l==p, l, p])
    plt.imshow(img)
    plt.show()

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
	 [[generation_dec/dense/BiasAdd/_167]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder_1':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/traitlets/config/application.

In [52]:
start_idx, end_idx = 0, 50

results = sess.run(outputs2, feed_dict={inputs:image_valid[start_idx:end_idx]})
print('정답 예측')
[[l==p, l, p] for l, p in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)])]

정답 예측


[[False, '애완동물', '여행'],
 [False, '일상', '여행'],
 [False, '애완동물', '여행'],
 [False, '셀카', '패션'],
 [False, '여행', '셀카'],
 [False, '여행', '연애'],
 [False, '애완동물', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '여행'],
 [True, '애완동물', '애완동물'],
 [False, '일상', '애완동물'],
 [False, '여행', '애완동물'],
 [False, '패션', '연애'],
 [True, '패션', '패션'],
 [False, '음식', '여행'],
 [False, '여행', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '애완동물'],
 [False, '음식', '셀카'],
 [False, '애완동물', '연애'],
 [False, '일상', '음식'],
 [True, '여행', '여행'],
 [False, '음식', '패션'],
 [False, '연애', '셀카'],
 [False, '셀카', '음식'],
 [False, '음식', '패션'],
 [False, '애완동물', '연애'],
 [False, '패션', '일상'],
 [False, '셀카', '애완동물'],
 [False, '일상', '패션'],
 [False, '음식', '여행'],
 [True, '패션', '패션'],
 [False, '패션', '일상'],
 [False, '일상', '연애'],
 [True, '연애', '연애'],
 [False, '애완동물', '패션'],
 [False, '일상', '연애'],
 [False, '애완동물', '패션'],
 [False, '애완동물', '패션'],
 [False, '연애', '여행'],
 [False, '셀카', '여행'],
 [True, '음식', '음식'],
 [False, '여행', '애완동물'],
 [True, '애완동물', '애완동물'],
 [Fal